In [1]:
import pandas as pd
import fnmatch

In [2]:
df = pd.read_csv("Resources/Parkingtickets_Dec2016.csv")
df.head()

,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Violation Time,Violation County,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Meter Number,Violation Post Code,Violation Description
0,GYG8911,NY,PAS,12/18/2016,24,SUBN,JEEP,1204P,Q,NaN,NaN,BK,2015,NaN,15 4,24-No Parking (exc auth veh)
1,8P82H,NY,OMT,12/21/2016,14,TAXI,NISSA,1015A,NY,ALL,ALL,YELLO,2014,-,NaN,NaN
2,GML6346,NY,PAS,12/19/2016,21,SDN,HONDA,1212P,K,1130A,0100A,GREY,2013,-,NaN,NaN
3,12203MG,NY,COM,12/1/2016,46,VAN,FRUEH,1137A,NY,ALL,ALL,BROWN,2015,-,NaN,NaN
4,HEG7444,NY,PAS,12/1/2016,36,4DSD,BMW,1027A,BK,NaN,NaN,BK,2016,NaN,NaN,PHTO SCHOOL ZN SPEED VIOLATION


In [3]:
for index, row in df.iterrows():
    try:
        if row["Vehicle Color"].startswith("D"):
            if row["Vehicle Color"].startswith("DK"):
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("DK")
            else:
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("D")
            
        elif row["Vehicle Color"].startswith("L"):
            if row["Vehicle Color"].startswith("LT"):
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("LT")
            else:
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("L")
        
    except AttributeError:
        df.drop(index, inplace=True)

In [4]:
# Finds rows with target colors
black = df['Vehicle Color'].str.startswith(("BLA", "BK"), na=False)
blue = df['Vehicle Color'].str.startswith("BL", na=False)
brown = df['Vehicle Color'].str.startswith(("BR", "BN"), na=False)
gray = df['Vehicle Color'].str.startswith(("GY", "GRA", "GREY", "GRY", "GEY", "S", "CH", "CR"), na=False)
green = df['Vehicle Color'].str.startswith(("GRE", "GN", "GRN"), na=False)
maroon = df['Vehicle Color'].str.startswith(("BRG", "BUR", "M"), na=False)
orange = df['Vehicle Color'].str.startswith("O", na=False)
red = df['Vehicle Color'].str.startswith("R", na=False)
tan = df['Vehicle Color'].str.startswith("T", na=False)
white = df['Vehicle Color'].str.startswith("W", na=False)
yellow = df['Vehicle Color'].str.startswith("Y", na=False)


# Re-defines colors with uniform abbreviations
df["Vehicle Color"].loc[black==True] = "BK"
df["Vehicle Color"].loc[blue==True] = "BL"
df["Vehicle Color"].loc[brown==True] = "BR"
df["Vehicle Color"].loc[gray==True] = "GY"
df["Vehicle Color"].loc[green==True] = "GN"
df["Vehicle Color"].loc[maroon==True] = "MN"
df["Vehicle Color"].loc[orange==True] = "OR"
df["Vehicle Color"].loc[red==True] = "RD"
df["Vehicle Color"].loc[tan==True] = "TN"
df["Vehicle Color"].loc[white==True] = "WH"
df["Vehicle Color"].loc[yellow==True] = "YE"

C:\Users\Elliott\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
print(df['Vehicle Color'].value_counts())

WH       7753
GY       7272
BK       7172
BL       3729
RD       1528
BR        954
GN        563
YE        526
GR        274
OR        182
TN        179
MN         84
G          54
GL         39
/          26
PR         22
GOLD       13
B          11
PURPL       8
UNKNO       6
UNK         4
BG          4
NOCL        4
BGE         3
PLE         2
GLBR        1
AVEN        1
XXX         1
NAVY        1
FREIG       1
P           1
NOC         1
CM          1
HONDA       1
BIEGE       1
CL          1
Name: Vehicle Color, dtype: int64
